In [13]:
import cv2
import numpy as np
import face_recognition
import firebase_admin
from firebase_admin import credentials, db, storage
import os

# Firebase initialization
cred = credentials.Certificate(r'C:\Users\HP\Desktop\REALTIME\serviceAccountKey.json')
firebase_admin.initialize_app(cred, {
    'databaseURL': "https://faceattendancerealtime-527cb-default-rtdb.firebaseio.com/",
    'storageBucket': 'faceattendancerealtime-527cb.firebasestorage.app'
}, name='RealTimeFaceRecognition91')

bucket = storage.bucket(app=firebase_admin.get_app('RealTimeFaceRecognition91'))

# Video capture and background image loading
cap = cv2.VideoCapture(0)
imgBackground = cv2.imread(r'C:\Users\HP\Desktop\REALTIME\Resources\background.png')

if imgBackground is None:
    print("Error: Failed to load the background image.")
    exit()

# Loading known faces and their encodings
path = r'C:\Users\HP\Desktop\REALTIME\Images'
images = []
studentIds = []
encodeListKnown = []

for file in os.listdir(path):
    if file.endswith('.jpg'):
        img = cv2.imread(f'{path}/{file}')
        images.append(img)
        studentIds.append(os.path.splitext(file)[0])

for img in images:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    encode = face_recognition.face_encodings(img)[0]
    encodeListKnown.append(encode)

print("Encoding Complete")

# Load mode images and resize to 633 x 1080
modePath = r'C:\Users\HP\Desktop\REALTIME\Resources\Modes'
modeImages = []

for file in os.listdir(modePath):
    if file.endswith('.png'):
        modeImage = cv2.imread(f'{modePath}/{file}')
        modeImage = cv2.resize(modeImage, (633, 1080))  # Resize to 633x1080
        modeImages.append(modeImage)

counter = 0
id = -1
studentInfo = {}
modeType = 0

while True:
    success, img = cap.read()
    if not success:
        print("Error: Failed to read from webcam.")
        break

    # Resize the webcam feed to fit the designated area (546x403)
    imgResized = cv2.resize(img, (546, 403))

    # Place the resized webcam feed into the background at the specified coordinates
    imgBackground[172:575, 24:570] = imgResized

    faceCurFrame = face_recognition.face_locations(imgResized)
    encodeCurFrame = face_recognition.face_encodings(imgResized, faceCurFrame)

    for encodeFace, faceLoc in zip(encodeCurFrame, faceCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)

        matchIndex = np.argmin(faceDis)
        if matches[matchIndex]:
            id = studentIds[matchIndex]
            if counter == 0:
                counter = 1
                modeType = 1

            # Fetch updated student info from the database whenever a match occurs
            studentInfo = db.reference(f'Students/{id}', app=firebase_admin.get_app('RealTimeFaceRecognition91')).get()
            print(studentInfo)

            # Retrieve and resize the student's image if available
            blob = bucket.get_blob(f'Images/{id}.jpg')
            if blob:
                array = np.frombuffer(blob.download_as_string(), np.uint8)
                imgStudent = cv2.imdecode(array, cv2.IMREAD_COLOR)
                imgStudent = cv2.resize(imgStudent, (203, 215))  # Resize to fit the target region (203x215)
                imgBackground[241:241+215, 733:733+203] = imgStudent
                
                #Update Data Of Attendance
                ref = db.reference(f'Students/{id}', app=firebase_admin.get_app('RealTimeFaceRecognition91'))
                studentInfo['total_attendance'] +=1
                ref.child('total_attendance').set(studentInfo['total_attendance'])

            # Overlay student info on the background
            cv2.putText(imgBackground, str(studentInfo.get('major', 'N/A')), (761, 580), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1)
            cv2.putText(imgBackground, str(id), (790, 527), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1)

            # Adjust name positioning to be centered within the given x range (773 to 946)
            (w, h), _ = cv2.getTextSize(studentInfo.get('name', 'N/A'), cv2.FONT_HERSHEY_COMPLEX, 1, 1)
            offset = (946 - 773 - w) // 2  # Center the text within the space
            cv2.putText(imgBackground, str(studentInfo.get('name', 'N/A')), (773 + offset, 216), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 1)

            counter += 1
        else:
            cv2.putText(imgBackground, "UNKNOWN FACE !!!", (773 + offset, 300), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2) 

    # Overlay the mode image in the designated area
    if modeType > 0 and modeType <= len(modeImages):
        imgBackground[1:608, 580:1080] = modeImages[modeType - 1]

    # Display the output
    cv2.imshow("Face Attendance", imgBackground)

    # Break the loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

Encoding Complete
{'last_attendance_time': '2025-01-15 17:47:34', 'major': 'Computer Science', 'name': 'Nwokocha Bethel Dan', 'standing': 'Good', 'starting year': 2020, 'total_attendance': 123, 'year': 4}
{'last_attendance_time': '2025-01-15 17:47:34', 'major': 'Computer Science', 'name': 'Nwokocha Bethel Dan', 'standing': 'Good', 'starting year': 2020, 'total_attendance': 124, 'year': 4}
{'last_attendance_time': '2025-01-15 17:47:34', 'major': 'Computer Science', 'name': 'Nwokocha Bethel Dan', 'standing': 'Good', 'starting year': 2020, 'total_attendance': 125, 'year': 4}
{'last_attendance_time': '2025-01-15 17:47:34', 'major': 'Computer Science', 'name': 'Nwokocha Bethel Dan', 'standing': 'Good', 'starting year': 2020, 'total_attendance': 126, 'year': 4}
{'last_attendance_time': '2025-01-15 17:47:34', 'major': 'Computer Science', 'name': 'Nwokocha Bethel Dan', 'standing': 'Good', 'starting year': 2020, 'total_attendance': 127, 'year': 4}
{'last_attendance_time': '2025-01-15 17:47:34',